In [1]:
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from transformers.models.auto.tokenization_auto import AutoTokenizer
import torch

In [2]:
testin = ["hello world!","second input:","third input:"]
testout = ["hi!!!!!!!!","yes youre second wow so great","now finished"]
tokenizer = AutoTokenizer.from_pretrained("/home/nova/cs336/assignment5-alignment/models/Qwen2.5-Math-1.5B")

In [3]:
def get_mask_tensor(io_len:list[tuple[int,int]])->torch.Tensor:
    """
    根据已知的io长度制造mask，只显露出模型输出的部分，mask掉输入和尾部padding
    """
    max_len = max([i + o for (i,o) in io_len])
    res = []
    for ilen, olen in io_len:
        #print("i,o:",ilen,olen)
        imask = [0] * ilen
        omask = [1] * olen
        padmask = [0] * (max_len - ilen - olen)
        res.append(imask + omask + padmask)
    rest = torch.tensor(res)
    return rest

In [4]:
def tokenize_prompt_and_output(
        prompt_strs:list[str],
        output_strs:list[str], 
        tokenizer:PreTrainedTokenizerBase ):
    """
    Tokenize the prompt and output strings, and construct a mask that is 1 for the response tokens and 0 for other tokens (prompt or padding).
    Args:
        prompt_strs(list[str]): List of prompt strings.
        output_strs(list[str]): List of output strings.
        tokenizer(PreTrainedTokenizer): Tokenizer to use for tokenization.
    Returns:
        output(dict[str, torch.Tensor]): Let prompt_and_output_lens be a list containing the lengths of the tokenized prompt and output strings. Then the returned dictionary should have the following keys.
        - input_ids: torch.Tensor of shape (batch_size, max(prompt_and_output_lens) - 1): the tokenized prompt and output strings, with the final token sliced off.
        - labels: torch.Tensor of shape (batch_size, max(prompt_and_output_lens) - 1): shifted input ids, i.e., the input ids without the first token.
        - response_mask: torch.Tensor of shape (batch_size, max(prompt_and_output_lens) - 1): a mask on the response tokens in the labels.
    """
    # print("输入:",prompt_strs)
    # print("输出:",output_strs)

    assert len(prompt_strs) == len(output_strs) , "输入与输出数量不等！"
    
    prompt_ids = [tokenizer.encode(s) for s in prompt_strs]
    #print(prompt_ids)

    response_ids = [tokenizer.encode(s) for s in output_strs]
    #print(response_ids)
    
    batch_ids = [p + r for p,r in zip(prompt_ids, response_ids)]
    #print(batch_ids)

    io_len = [(len(lp),len(lr)) for lp, lr in zip(prompt_ids, response_ids)]
    #print(io_len)

    max_len = max([len(io) for io in batch_ids])
    #print("最长的序列长度：",max_len)

    padding_id = tokenizer.pad_token_id
    #print("padding id:",padding_id)
    #print(tokenizer.decode(padding_id))

    for io in batch_ids:
        l = len(io)
        pad = [padding_id for _ in range(max_len - l)]
        io += pad

    #print(batch_ids)

    



    padded_batch_tensor = torch.tensor(batch_ids)
    #print(padded_batch_tensor)

    mask_tensor = get_mask_tensor(io_len)
    #print(mask_tensor)

    res =  {
        "input_ids":padded_batch_tensor[:,:-1],
        "labels":padded_batch_tensor[:,1:],
        "response_mask":mask_tensor[:,1:]
    }

    return res

tokenize_prompt_and_output(testin,testout,tokenizer)

{'input_ids': tensor([[ 14990,   1879,      0,   6023,  50667, 151643, 151643, 151643],
         [  5569,   1946,     25,   9693,  70075,   2086,  35665,    773],
         [ 31727,   1946,     25,   3328,   8060, 151643, 151643, 151643]]),
 'labels': tensor([[  1879,      0,   6023,  50667, 151643, 151643, 151643, 151643],
         [  1946,     25,   9693,  70075,   2086,  35665,    773,   2244],
         [  1946,     25,   3328,   8060, 151643, 151643, 151643, 151643]]),
 'response_mask': tensor([[0, 0, 1, 1, 0, 0, 0, 0],
         [0, 0, 1, 1, 1, 1, 1, 1],
         [0, 0, 1, 1, 0, 0, 0, 0]])}

### Problem (compute_entropy): Per-token entropy (1 point)

**Deliverable:** Implement a method `compute_entropy` that computes the per-token entropy of next-token predictions. The following interface is recommended:

```python
def compute_entropy(logits: torch.Tensor) -> torch.Tensor:
```
> Get the entropy of the next-token predictions (i.e., entropy over the vocabulary dimension).

- **Args:**
    - `logits: torch.Tensor`: Tensor of shape `(batch_size, sequence_length, vocab_size)` containing unnormalized logits.
- **Returns:**
    - `torch.Tensor`: Shape `(batch_size, sequence_length)`. The entropy for each next-token prediction.

**Note:** you should use a numerically stable method (e.g., using `logsumexp`) to avoid overflow.

To test your code, implement `adapters.run_compute_entropy`. Then run `uv run pytest -k test_compute_entropy` and ensure your implementation passes.


In [27]:
import einops
from jaxtyping import Float, Int, Bool

def compute_entropy(
    logits: Float[torch.Tensor,"batch_size sequence_length vocab_size"]
) -> Float[torch.Tensor,"batch_size sequence_length"]:
    """
    Get the entropy of the next-token predictions (i.e., entropy over the vocabulary dimension).
    Args:
        logits(torch.Tensor): Tensor of shape `(batch_size, sequence_length, vocab_size)` containing unnormalized logits.
    Returns:
        output(torch.Tensor): Shape `(batch_size, sequence_length)`. The entropy for each next-token prediction.
    """
    # 每个位置取对数，乘以自己的相反数，然后沿最后一维求和
    print("输入：",logits)

    probs = logits.softmax(-1)
    


    neglogp = - torch.log(probs)
    print("负对数：",neglogp)
    ent = einops.einsum(logits, neglogp, "b s v , b s v -> b s v")
    print("自身乘以负对数：",ent)
    res = einops.reduce(ent, "b s v -> b s", "sum")
    print("结果：",res)
    return res

In [28]:
a = torch.tensor([1,2,3])
a

tensor([1, 2, 3])

In [29]:
- torch.log(a)

tensor([-0.0000, -0.6931, -1.0986])

In [30]:
import numpy as np

data = np.load("/home/nova/cs336/assignment5-alignment/tests/_snapshots/test_compute_entropy.npz")
print(data.files)  

for key in data.files:
    print(f"{key}: {data[key].shape}")
    print(data[key])

['array']
array: (2, 10)
[[4.1573386 4.25809   4.2000027 4.1944656 4.106727  4.0669866 4.090223
  4.0061703 4.149583  4.099516 ]
 [4.282163  3.8720474 4.1143    4.0242934 4.2255774 4.0835614 4.2772403
  4.0522842 4.2011576 4.273294 ]]


In [31]:
torch.manual_seed(42)
inputs = torch.randn(size=(2,10,100))
compute_entropy(inputs)

输入： tensor([[[ 1.9269,  1.4873,  0.9007,  ...,  0.4880,  0.7846,  0.0286],
         [ 0.6408,  0.5832,  1.0669,  ...,  0.3581,  0.4788,  1.3537],
         [ 0.5261,  2.1120, -0.5208,  ...,  0.2539,  0.9364,  0.7122],
         ...,
         [-1.2520,  3.0250,  1.3463,  ...,  1.4162,  0.6834, -0.1383],
         [ 0.9213,  0.5282, -0.0082,  ..., -0.3867,  0.9578, -0.8225],
         [-2.3908,  0.3222,  1.8754,  ..., -0.1493, -0.5523, -0.0934]],

        [[-1.0284,  0.4044,  2.1426,  ..., -0.0563, -1.4897, -1.5195],
         [ 0.3258, -1.4584,  1.8989,  ...,  1.7561,  0.2113,  1.4860],
         [ 0.5585,  0.3491,  0.8484,  ...,  0.0911,  0.6719,  0.9852],
         ...,
         [-0.6106,  1.0629,  1.2222,  ..., -0.3292, -0.1140, -0.8452],
         [ 0.3004,  1.6395, -1.0744,  ...,  0.3046, -0.7002,  1.7811],
         [-0.2937,  0.5243,  1.0186,  ..., -1.2133,  0.9745,  0.4532]]])
负对数： tensor([[[3.2135, 3.6531, 4.2397,  ..., 4.6524, 4.3558, 5.1118],
         [4.4473, 4.5048, 4.0211,  ..., 4.

tensor([[ -90.8289,  -50.5721,  -68.7912,  -85.5457,  -90.7824, -151.5848,
          -79.0051,  -73.5918, -182.0801, -132.6149],
        [ -13.7322,  -47.4912,  -98.1791, -140.6179,  -82.2795, -179.6641,
          -73.4825, -135.3148, -121.5303,  -40.8543]])

In [32]:
test = torch.randn(size=(1,2,3))
print(test)
compute_entropy(test)

tensor([[[ 1.0556,  1.4968,  0.3602],
         [-1.2984,  0.3366, -0.5514]]])
输入： tensor([[[ 1.0556,  1.4968,  0.3602],
         [-1.2984,  0.3366, -0.5514]]])
负对数： tensor([[[1.1163, 0.6751, 1.8117],
         [2.1090, 0.4740, 1.3620]]])
自身乘以负对数： tensor([[[ 1.1784,  1.0104,  0.6526],
         [-2.7382,  0.1596, -0.7510]]])
结果： tensor([[ 2.8414, -3.3296]])


tensor([[ 2.8414, -3.3296]])